In [1]:
import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import time
import pickle
from sklearn.metrics import average_precision_score

import resnetw

ModuleNotFoundError: ignored

In [0]:
def ctime():
    if torch.cuda.is_available(): torch.cuda.synchronize()
    return(time.time())

In [0]:
train_batch_size = 125
test_batch_size = 400
verbose = 0
epochs = 132
pos_weight = 18  # 18 for CIFAR10 and 10 for CIFAR100
alpha = 0.9
threshold = 1.4
weight_decay = 0.0001
dataset = "CIFAR10"
lr0 = 0.1
width = 64

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(device)

cuda:0


In [0]:
# https://gist.github.com/weiaicunzai/e623931921efefd4c331622c344d8151
# SVHN: mean=[0.4377, 0.4438, 0.4728], std=[0.1980, 0.2010, 0.1970]
# MNIST: mean=[0.1307], std=[0.3081]
# FashionMNIST: mean=[0.2860], std=[0.3530]
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

def getSetsClasses(dataset, root='./data'):
    if (dataset == 'CIFAR10'):
        normalize = transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
        transform_train = transforms.Compose(
            [transforms.RandomHorizontalFlip(),
             transforms.RandomCrop(32, 4),
             transforms.ToTensor(), normalize])
        transform_test = transforms.Compose(
            [transforms.ToTensor(), normalize])
        trainset = torchvision.datasets.CIFAR10(
            root=root, train=True, download=True, transform=transform_train)
        testset = torchvision.datasets.CIFAR10(
            root=root, train=False, download=True, transform=transform_test)
        with open(root+'/cifar-10-batches-py/batches.meta', 'rb') as f:
            classes = pickle.load(f)["label_names"]
        return trainset, testset, classes
    if (dataset == 'CIFAR100'):
        normalize = transforms.Normalize(
            mean=[0.5071, 0.4865, 0.4409], std=[0.2673, 0.2564, 0.2762])
        transform_train = transforms.Compose(
            [transforms.RandomHorizontalFlip(),
             transforms.RandomCrop(32, 4),
             transforms.ToTensor(), normalize])
        transform_test = transforms.Compose(
            [transforms.ToTensor(), normalize])
        trainset = torchvision.datasets.CIFAR100(
            root=root, train=True, download=True, transform=transform_train)
        testset = torchvision.datasets.CIFAR100(
            root=root, train=False, download=True, transform=transform_test)
        with open(root+'/cifar-100-python/meta', 'rb') as f:
            classes = pickle.load(f)["fine_label_names"]
        return trainset, testset, classes
    return None

trainset, testset, classes = getSetsClasses(dataset)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=train_batch_size, shuffle=True, num_workers=8, pin_memory=True)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=test_batch_size, shuffle=False, num_workers=8, pin_memory=True)
num_classes = len(classes)
print(num_classes, "classes")

Files already downloaded and verified
Files already downloaded and verified
10 classes


In [0]:
def getScoresLabels(net, loader, device):
    # Not memory efficient
    lscores = []
    llabels = []
    net.eval()
    with torch.no_grad():
        for data in loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = net(inputs)
            lscores.append(outputs)
            llabels.append(F.one_hot(labels, torch.tensor(outputs.size())[1].item()))
    return torch.cat(lscores), torch.cat(llabels)

def getRates(scores, labels, threshold = 0):
    num_classes = torch.tensor(scores.size())[1].item()
    rates = torch.zeros(4, num_classes, dtype=torch.int).to(device)
    predicted = (scores > threshold).int()
    # true negatives, false negative, false positive, true positive
    rates[0] = torch.sum(((predicted == 0) & (labels == 0)).int(), dim=0)
    rates[1] = torch.sum(((predicted == 0) & (labels == 1)).int(), dim=0)
    rates[2] = torch.sum(((predicted == 1) & (labels == 0)).int(), dim=0)
    rates[3] = torch.sum(((predicted == 1) & (labels == 1)).int(), dim=0)
    return rates

def accuracy(scores, labels):
    num_classes = torch.tensor(scores.size())[1].item()
    class_correct = list(0. for i in range(num_classes))
    class_total = list(0. for i in range(num_classes))
    predicted = torch.argmax(scores, 1)
    truelabel = torch.argmax(labels, 1)
    c = (predicted == truelabel).squeeze()
    for i in range(list(scores.shape)[0]):
        label = truelabel[i]
        class_correct[label] += c[i].item()
        class_total[label] += 1
    return class_correct, class_total

In [0]:
class LeNet(nn.Module):
    def __init__(self, w = 32, h = 32, i = 3, n = [6, 16, 120, 84], o = 10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(i, n[0], 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(n[0], n[1], 5)
        self.fc1 = nn.Linear(n[1] * (w//4-3) * (h//4-3), n[2])
        self.fc2 = nn.Linear(n[2], n[3])
        self.fc3 = nn.Linear(n[3], o)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, torch.prod(torch.tensor(list(x.size()[1:]))).item())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [0]:
def evaluate(net, loader, device, set = 'Test'):
    t0 = ctime()
    scores, labels = getScoresLabels(net, loader, device)
    # class_correct, class_total = accuracy(scores, labels)
    class_correct, class_total = accuracy(scores, labels)
    print('%s time: %3.2fs' % (set, ctime()-t0), end = "  ")
    print('Accuracy: %2.2f %%' % (100 * sum(class_correct) / sum(class_total)), end = "  ")
    labels = labels.to("cpu").numpy()
    scores = scores.to("cpu").numpy()
    map = torch.tensor(average_precision_score(labels, scores, average=None)).float().mean().item()
    print('%s MAP: %2.2f %%' % (set, 100 * map))

In [0]:
t0 = ctime()
#net = LeNet(n=[32, 32, 240, 120], o=num_classes).to(device)
net = resnetw.ResNet(resnetw.BasicBlock, [3, 3, 3], num_classes=num_classes, width=width).to(device)
print('Init time:   %3.2fs' % (ctime()-t0), end = "  ")
evaluate(net, testloader, device)

Init time:   0.08s  Test time: 2.35s  Accuracy: 10.00 %  Test MAP: 11.36 %


In [0]:
criterionMC = nn.CrossEntropyLoss()
criterionML = nn.BCEWithLogitsLoss(pos_weight=pos_weight*torch.ones([num_classes]).to(device))
optimizer = optim.SGD(net.parameters(), lr=lr0, momentum=0.9, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[epochs//2,(3*epochs)//4])

In [0]:
def train(epoch, net, trainloader, device, optimizer, scheduler, criterionMC, criterionML, alpha):
    print('Epoch: %3d' % epoch, end = "  ")
    running_loss = 0.0
    t0 = ctime()
    net.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        if (alpha <= 0): loss = criterionMC(outputs, labels)
        elif (alpha >= 1): loss = criterionML(outputs, F.one_hot(labels,num_classes).type_as(outputs))
        else: loss = (1-alpha)*criterionMC(outputs, labels) + \
            alpha*criterionML(outputs, F.one_hot(labels,num_classes).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Train time: %3.2fs' % (ctime()-t0), end = "  ")
    print('Train loss: %5.4f' % (running_loss*train_batch_size/len(trainset)), end = "  ")    

In [0]:
t1 = ctime()
for epoch in range(epochs):  # loop over the dataset multiple times
    train(epoch, net, trainloader, device, optimizer, scheduler, criterionMC, criterionML, alpha)
    evaluate(net, testloader, device)
    scheduler.step()
tt = ctime()-t1
print('Finished Training, total time %4.2fs' % (tt))

Epoch:   0  Train time: 24.56s  Train loss: 1.3669  Test time: 2.01s  Accuracy: 40.46 %  Test MAP: 44.08 %
Epoch:   1  Train time: 24.74s  Train loss: 0.9291  Test time: 2.03s  Accuracy: 54.14 %  Test MAP: 63.22 %
Epoch:   2  Train time: 24.87s  Train loss: 0.7494  Test time: 2.04s  Accuracy: 64.56 %  Test MAP: 74.51 %
Epoch:   3  Train time: 24.90s  Train loss: 0.6207  Test time: 2.03s  Accuracy: 74.92 %  Test MAP: 82.24 %
Epoch:   4  Train time: 24.96s  Train loss: 0.5358  Test time: 2.03s  Accuracy: 73.40 %  Test MAP: 82.86 %
Epoch:   5  Train time: 25.02s  Train loss: 0.4663  Test time: 2.04s  Accuracy: 77.20 %  Test MAP: 86.21 %
Epoch:   6  Train time: 25.00s  Train loss: 0.4162  Test time: 2.04s  Accuracy: 77.21 %  Test MAP: 88.32 %
Epoch:   7  Train time: 25.01s  Train loss: 0.3811  Test time: 2.03s  Accuracy: 67.86 %  Test MAP: 85.49 %
Epoch:   8  Train time: 25.00s  Train loss: 0.3429  Test time: 2.03s  Accuracy: 83.94 %  Test MAP: 91.91 %
Epoch:   9  Train time: 24.99s  Train

In [0]:
t0 = ctime()
scores, labels = getScoresLabels(net, testloader, device)
class_correct, class_total = accuracy(scores, labels)
print('Test time: %3.2fs' % (ctime()-t0))

print('Overall accuracy  : %2.2f %%' % (
    100 * sum(class_correct) / sum(class_total)))

for i in range(num_classes):
    print('Accuracy of %5s : %2.2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Test time: 2.05s
Overall accuracy  : 94.56 %
Accuracy of airplane : 95.60 %
Accuracy of automobile : 97.60 %
Accuracy of  bird : 91.90 %
Accuracy of   cat : 89.80 %
Accuracy of  deer : 96.70 %
Accuracy of   dog : 89.70 %
Accuracy of  frog : 96.50 %
Accuracy of horse : 95.70 %
Accuracy of  ship : 96.00 %
Accuracy of truck : 96.10 %


In [0]:
scores, labels = getScoresLabels(net, testloader, device)
rates = getRates(scores, labels, threshold = threshold)
print("true negatives, false negative, false positive, true positive")
print(rates)
cor = rates[3].float()
pre = cor/(rates[3]+rates[2])
rec = cor/(rates[3]+rates[1])
print(pre)
print(pre.mean().item())
print(rec)
print(rec.mean().item())
labels = labels.to("cpu").numpy()
scores = scores.to("cpu").numpy()
ap = torch.tensor(average_precision_score(labels, scores, average=None)).float()
print(ap)
map = ap.mean().item()
print(map)

true negatives, false negative, false positive, true positive
tensor([[8937, 8967, 8949, 8863, 8936, 8928, 8971, 8981, 8966, 8962],
        [  44,   21,   92,  101,   31,  103,   37,   41,   40,   38],
        [  63,   33,   51,  137,   64,   72,   29,   19,   34,   38],
        [ 956,  979,  908,  899,  969,  897,  963,  959,  960,  962]],
       device='cuda:0', dtype=torch.int32)
tensor([0.9382, 0.9674, 0.9468, 0.8678, 0.9380, 0.9257, 0.9708, 0.9806, 0.9658,
        0.9620], device='cuda:0')
0.946302056312561
tensor([0.9560, 0.9790, 0.9080, 0.8990, 0.9690, 0.8970, 0.9630, 0.9590, 0.9600,
        0.9620], device='cuda:0')
0.9451999664306641
tensor([0.9864, 0.9931, 0.9785, 0.9514, 0.9880, 0.9604, 0.9932, 0.9940, 0.9929,
        0.9911])
0.9829050302505493


In [0]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [0]:
print(net)
psize = []
params = 0
flops = 0
wh = [28, 10, 1, 1, 1]

#for param in list(net.parameters()):
#    psize.append(list(param.shape))

#for i in range(len(psize)//2):
#    weight = torch.prod(torch.tensor(psize[2*i])).item()
#    bias = psize[2*i+1][0]
#    flop = 2*weight*wh[i]**2
#    print(psize[2*i], psize[2*i+1], weight,"+",bias, ',', flop)
#    params += weight+bias
#    flops += flop

#print(params, ',', flops)
#print('FP performance: %3.2f Gflops' %
#      (flops*(2*len(trainset)+len(testset))*epochs/1000000000/tt))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [0]:
import numpy as np
net_parameters = filter(lambda p: p.requires_grad, net.parameters())
params = sum([np.prod(p.size()) for p in net_parameters])
print(params)

4286026


In [0]:
out = torch.zeros(2,4)
torch.tensor(out.size())[1].item()

4